In [8]:
import pandas as pd
human_data = pd.read_table("data/human_text_indo_v2.txt", 
                           header=None,
                           engine='python')
human_data.columns = ["human"]
robot_data = pd.read_table("data/robot_text_indo.txt",
                           header=None,
                           engine='python')
robot_data.columns = ["robot"]

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
robot_data.head()

In [ ]:
data = {'human':human_data.human, 'robot': robot_data.robot}
df = pd.DataFrame(data)
df.head(20)

In [ ]:
# ganti [...] dgn kata hai
import re
df.human = df.human.apply(lambda x : re.sub(r"\[\w+\]", "hai", x))
df.robot = df.robot.apply(lambda x : re.sub(r"\[\w+\]", "hai", x))

In [ ]:
# konversi huruf kecil
df.human = df.human.apply(lambda x: x.lower())
df.robot = df.robot.apply(lambda x: x.lower())

In [ ]:
# hapus tanda baca
import string
exclude = set(string.punctuation)
df.human = df.human.apply(lambda x : ''.join(ch for ch in x if ch not in exclude))
df.robot = df.robot.apply(lambda x : ''.join(ch for ch in x if ch not in exclude))


In [ ]:
# hapus angka
remove_digits = str.maketrans('', '', string.digits)
df.human = df.human.apply(lambda x: x.translate(remove_digits))
df.robot = df.robot.apply(lambda x: x.translate(remove_digits))


In [ ]:
# hapus emoticon
df.human = df.human.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
df.robot = df.robot.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))


In [ ]:
df.head()

In [ ]:
# split data
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.1)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
# membuat vocab (human & robot) dari data train
vocabulary = set()

# menggabungkan robot kolom dgn human kolom
for idx, row in df_train.iterrows():
    sent = row.human + '' + row.robot
    [vocabulary.update(sent.split())]

print(f"Ukuran Vocab : {len(vocabulary)}")


In [ ]:
all_vocab = []

for idx, row in df_train.iterrows():
    sent = row.human + '' + row.robot
    [all_vocab.append(i) for i in sent.split()]
print(f"Jumlah semua token : {len(all_vocab)}")

In [ ]:
# masukan token >> vocab >> counter >> membentuk dict. (vocab: frekuensi) >> 
# ..sort yg lebih dr threshold >> ambil x0 (q dari sorted dict) 

# hitung frekuensi vocab dan hapus yg tidak perlu (sedikit)
from collections import Counter
counter = Counter(all_vocab)

dic_ = dict(counter)

# jika vocab <3 maka tdk dipakai utk proses selanjutnya (eliminasi vocab)
# data jumlah vocab diperlukan utk one.dot encoding sblm embedding
threshold = 3

sorted_dic = sorted(dic_.items(), reverse=True, key = lambda x: x[1])
sorted_dic = [x for x in sorted_dic if x[1] > threshold]
all_vocab = [x[0] for x in sorted_dic]
len(all_vocab)

In [ ]:
# buat dictionary word_to_idx & idx_to_word sblm masuk embedding
# machine learning hanya bisa ambil angka bukan kata
ix = 1
word_to_idx = {}
idx_to_word = {}

# e = kata
for e in all_vocab:
    word_to_idx[e] = ix
    idx_to_word[ix] = e
    ix += 1


In [ ]:
# data komutatif
word_to_idx['nama']

In [ ]:
idx_to_word[99]

In [ ]:
# tambahkan "startseq" dan "endseq" (string awal kalimat dan akhir kalimat)

word_to_idx['startseq'] = 846
word_to_idx['startseq'] = 847

idx_to_word[846] = 'startseq'
idx_to_word[847] = 'endseq'

In [ ]:
# update vocab_size
vocab_size = len(idx_to_word) + 1
vocab_size

In [ ]:
# tambahkan startseq & endseq di data training (hanya utk robot >> label)
df_train.robot = df.robot.apply(lambda x: 'startseq '+ x + 'endseq')
df_train.head()

In [ ]:
# membuat generator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
def data_generator(train_df, word_to_idx, max_len, number_conversation):
    x1, x2, y = [], [], []
    n = 0
    # ekstrak input berasal dari kolom human >> diiterasi >> setiap conversation diubah ke kata ke index
    while True:
        for idx, row in train_df.iterrows():
            seq_human = [ word_to_idx[word] for word in row['human'].split() if word in word_to_idx]
            seq_human = pad_sequences([seq_human], maxlen=max_len, value=0, padding='post')[0]
            seq_robot = [ word_to_idx[word] for word in row['robot'].split() if word in word_to_idx]

            for i in range(1, len(seq_robot)):
                in_seq = seq_robot[:i]
                out_seq = seq_robot[i]

                in_seq = pad_sequences([in_seq], maxlen=max_len, value=0, padding='post')[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                X1.append(seq_human)
                X2.append(in_seq)
                y.append(out_seq)
            if n==number_conversation:
                # [X1, X2], y
                yield([np.array(X1), np.array(X2)], np.array(y))
                X2, X2, y = [], [], []
                

In [ ]:
datasample = df_train.sample(2)
datagen = data_generator(datasample, word_to_idx, 50, len(datasample))

In [ ]:
# tes generator
datasample

In [ ]:
# training 
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, LSTM, Add
from tensorflow.keras.models import Model, load_model
max_len = 50
input_chat = Input(shape=(max_len,))
input_x = Embedding(input_dim=vocab_size, outout_dim=50, mask_zero=True)(input_chat)
input_x = Dropout(0.3)(input_x)
input_x = LSTM(256)(input_x)

In [ ]:
output_x = Input(shape=(max_len,))
output_x = Embedding(input_dim=vocab_size, output_dim=50, mask_zero=True)(output_chat)
output_x = Dropout(0.3)(output_x)
output_x = LSTM(256)(output_x)